In [2]:
from sqlalchemy import create_engine
import pandas as pd
from random import randint, choice
import time
import uuid
import json


rows = []
for _ in range(1):  # Assuming you're creating one row for demonstration
    events = [
        {"event_type": "login", "timestamp": time.time()},
        {"event_type": "view_item", "timestamp": time.time() + randint(1, 600)},
        {"event_type": "add_to_cart", "timestamp": time.time() + randint(600, 1200)},
        {"event_type": "checkout", "timestamp": time.time() + randint(1200, 1800)},
    ]
    row = {
        "user_id": str(uuid.uuid4()),
        "device_id": str(uuid.uuid4()),
        "timestamp": time.time(),
        "session_duration": randint(1, 600),
        "clicks": randint(1, 100),
        "views": randint(1, 100),
        "location_info": {
            "country": choice(["USA", "Canada", "Germany", "France", "Japan"]),
            "city": choice(["New York", "Toronto", "Berlin", "Paris", "Tokyo"]),
            "latitude": round(34.0522 + randint(-100, 100) / 1000.0, 4),
            "longitude": round(-118.2437 + randint(-100, 100) / 1000.0, 4),
        },
        "device_type": choice(["smartphone", "tablet", "desktop", "laptop"]),
        "operating_system": choice(["iOS", "Android", "Windows", "macOS", "Linux"]),
        "app_version": f"{randint(1,5)}.{randint(0,9)}.{randint(0,9)}",
        "events": json.dumps(events)  # Serialize the events list to a JSON string
    }
    rows.append(row)

# Create DataFrame
df = pd.DataFrame(rows)

host = "localhost"
database = "postgres"
username = "postgres"
password = "password"
port = "5432"

# Create a connection to the database
engine = create_engine(
    f'postgresql://{username}:{password}@{host}:{port}/{database}'
)

# convert location_info to string
df['location_info'] = df['location_info'].apply(str)

# # Insert into PostgreSQL with pandas
df.to_sql('user_sessions', engine, if_exists='append', index=False, schema='analytics', method='multi')
print("Data inserted successfully")

Data inserted successfully


In [4]:
from minio import Minio

MINIO_ENDPOINT = 'localhost:9000'
MINIO_ACCESS_KEY = 'admin'
MINIO_SECRET_KEY = 'password'

minioClient = Minio(MINIO_ENDPOINT,
                            access_key=MINIO_ACCESS_KEY,
                            secret_key=MINIO_SECRET_KEY,
                            secure=False)

# get data from analytics-raw bucket
objects = minioClient.list_objects('analytics-raw-data', recursive=True)

import boto3
import io
import pandas as pd

# # Read the parquet file
# buffer = io.BytesIO()
# s3 = boto3.resource('s3')
# object = s3.Object('my-bucket-name','path/to/parquet/file')
# object.download_fileobj(buffer)
# df = pd.read_parquet(buffer)

# print(df.head())

df = pd.DataFrame()
for obj in objects:
    # get data from file
    data = minioClient.get_object('analytics-raw-data', obj.object_name)

    data = io.BytesIO(data.read())
    data = pd.read_parquet(data)
    df = pd.concat([df, data])

df.head()

,user_id,device_id,timestamp,session_duration,clicks,views,location_info,device_type,operating_system,app_version,events
0,d2ba1d59-4ceb-48fe-9040-3e40458229be,c143f8f3-7c78-4df2-aa7f-6ec3c951963c,1.720385e+09,318,40,17,"{'city': 'New York', 'country': 'USA', 'latitu...",smartphone,macOS,5.7.0,"[{'event_type': 'login', 'timestamp': 17203850..."


In [38]:
import kafka 
from kafka.admin import KafkaAdminClient, NewTopic


consumer = kafka.KafkaConsumer(group_id='dev', bootstrap_servers='localhost:9092')
print(consumer.topics())
# print(consumer.partitions_for_topic('test'))



{'analytics_raw_data', 'sample'}


In [41]:
from kafka import KafkaAdminClient

kafka_admin_client_config = {
    "bootstrap_servers": "localhost:9092",
    "client_id": "dev",
}

admin = KafkaAdminClient(**kafka_admin_client_config)
kafka_topics = admin._client.cluster.topics()

NodeNotReadyError: NodeNotReadyError